In [4]:
import pandas as pd
import numpy as np
import datarobot as dr
import os
import requests
pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 200

USERNAME = os.environ['DATAROBOT_USERNAME']
API_KEY = os.environ['DATAROBOT_API_TOKEN']
DATAROBOT_KEY = os.environ['DATAROBOT_KEY']
API_URL = 'https://mlops.dynamic.orm.datarobot.com/predApi/v1.0/deployments/{deployment_id}/predictions'
ENDPOINT = 'https://app.datarobot.com/api/v2'

dr.Client(token=API_KEY, endpoint=ENDPOINT)  

In [10]:
# Work with a lending club deployment
deployments_lc = dr.Deployment.list(search='is_bad Predictions')  # Grab a deployment from a project built from
                                                                  # a lending club demo dataset
# Get the most recent one
deployment = deployments_lc[0]

DEPLOYMENT_ID = deployment.id
PROJECT_ID = deployment.model.get('project_id')

PRED_FILE = './data/DR_Demo_10K_Lending_Club_Loans_pred.csv'

df_preds = pd.read_csv(PRED_FILE)

In [11]:
dep = dr.Deployment.get(DEPLOYMENT_ID)
prj = dr.Project.get(PROJECT_ID)

In [15]:
dep.id

'611bffb30e1f9b27fec90821'

In [22]:
API_URL.format(deployment_id=deployment.id)

'https://mlops.dynamic.orm.datarobot.com/predApi/v1.0/deployments/611bffb30e1f9b27fec90821/predictions'

In [23]:
class DataRobotPredictionError(Exception):
    """Raised if there are issues getting predictions from DataRobot"""


def make_datarobot_deployment_predictions(data, deployment_id, content_type='json'):
    if content_type == 'json':
        ctype = 'application/json; charset=UTF-8'
        data = data.to_json(orient='records')
    else:  # if from input file
        ctype = 'text/plain; charset=UTF-8'

    # Set HTTP headers. The charset should match the contents of the file.
    headers = {
        'Content-Type': ctype,
        'Authorization': 'Bearer {}'.format(API_KEY),
        'DataRobot-Key': DATAROBOT_KEY,
    }

    url = API_URL.format(deployment_id=deployment_id)
    
    print(url)

    # Prediction Explanations:
    # See the documentation for more information:
    # https://app.datarobot.com/docs/users-guide/predictions/api/new-prediction-api.html#request-pred-explanations
    # Should you wish to include Prediction Explanations or Prediction Warnings in the result,
    # Change the parameters below accordingly, and remove the comment from the params field below:

    params = {
        # If explanations are required, uncomment the line below
        # 'maxExplanations': 3,
        # 'thresholdHigh': 0.5,
        # 'thresholdLow': 0.15,
        # Uncomment this for Prediction Warnings, if enabled for your deployment.
        # 'predictionWarningEnabled': 'true',
    }
    # Make API request for predictions
    predictions_response = requests.post(
        url,
        data=data,
        headers=headers,
        # Prediction Explanations:
        # Uncomment this to include explanations in your prediction
        # params=params,
    )
    _raise_dataroboterror_for_status(predictions_response)
    # Return a Python dict following the schema in the documentation
    return predictions_response.json()


def _raise_dataroboterror_for_status(response):
    """Raise DataRobotPredictionError if the request fails along with the response returned"""
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError:
        err_msg = '{code} Error: {msg}'.format(
            code=response.status_code, msg=response.text)
        raise DataRobotPredictionError(err_msg)

In [24]:
make_datarobot_deployment_predictions(df_preds.iloc[:2], DEPLOYMENT_ID)

https://mlops.dynamic.orm.datarobot.com/predApi/v1.0/deployments/611bffb30e1f9b27fec90821/predictions


{'data': [{'rowId': 0,
   'prediction': 0.0,
   'predictionThreshold': 0.5,
   'predictionValues': [{'label': 1, 'value': 0.2542051034},
    {'label': 0.0, 'value': 0.7457948966}],
   'deploymentApprovalStatus': 'APPROVED'},
  {'rowId': 1,
   'prediction': 0.0,
   'predictionThreshold': 0.5,
   'predictionValues': [{'label': 1, 'value': 0.3023641906},
    {'label': 0.0, 'value': 0.6976358094}],
   'deploymentApprovalStatus': 'APPROVED'}]}

In [14]:
df_preds.iloc[0].to_json(orient="records")

'[251913022771,2925428815144927,2019121512440340,null,null,"a251L0000043HSqQAM",178442.0,178442.0,true,"a2F1L0000053RrWUAU",null,null,false,false,25,"Residential",2000.0,"ATT_CELLULAR_LTE","TX","Other",7875,"Outdoor Camera","Camera Offline",null,"Summer Service",19.0,3.0,28812.0,19.0,3.0,28812.0,null,46.0,"2019-12-15",0,0,0,"NIS",null,"White","api_user_ecommerce","Smart Home",1781.0,1.0,22771,25.0,2019,13,3,303,46,30.402641,-97.6486,3,49,289.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,137.0,600.0,1975.0,44.0,3.0,49.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,null,n